## PARAMS & overview
Ho centralizzato i path e definito gli output principali per preparare i file destinati a Tableau.
Uso questo notebook per: caricare il file cleaned, applicare la rimozione outlier sul prezzo (IQR), creare il file riga-per-riga per la mappa e l'aggregato per dealer_region.

In [1]:
# === PARAMS · PATH CANONICI (repo-aware, cross-platform) ===
# Scopo (per il team):
# - Evitare path assoluti e dipendenze dalla cartella di esecuzione.
# - Avere path RELATIVI ma robusti, ancorati alla root del repository.
# - Standardizzare dove leggere/scrivere i file (raw/processed/mappings).
#
# Nota: se spostiamo il notebook (es. /notebook → /), la funzione get_repo_root()
# trova comunque la root corretta cercando la cartella "data" nei livelli superiori.

import os
import pandas as pd
import numpy as np
from pathlib import Path

def get_repo_root():
    """Trova la root del repo cercando 'data/' nei livelli correnti/superiori."""
    here = Path.cwd().resolve()
    for base in (here, here.parent, here.parent.parent):
        if (base / "data").exists():
            return base
    return here  # fallback: se non trova 'data', usa la CWD (ambiente atipico)

# Directory di riferimento del progetto (usate in TUTTO il notebook)
REPO_ROOT     = get_repo_root()                   # es. .../PROGETTO_FINALE
RAW_DIR       = REPO_ROOT / "data" / "raw"        # input grezzi (non toccati)
PROCESSED_DIR = REPO_ROOT / "data" / "processed"  # output puliti/pronti per Tableau
MAPPINGS_DIR  = REPO_ROOT / "notebook" / "mappings"  # lookup/patch di supporto

# File “canonici” che useremo in questo notebook:
# - CLEANED_PATH: dataset pulito e normalizzato generato dai notebook di preprocessing (Matteo)
# - OUT_AGG_PATH: export aggregato per dealer_region
# - OUT_CITY_PATH: export con lookup città/stato per geocodifica in Tableau
CLEANED_PATH  = PROCESSED_DIR / "database_cleaned_2.csv"            # << file sorgente corretto
OUT_AGG_PATH  = PROCESSED_DIR / "agg_by_dealer_region_for_tableau.csv"
OUT_CITY_PATH = PROCESSED_DIR / "database_for_tableau_city_state.csv"

# Log diagnostico (utile se qualcosa “non si trova”)
print("REPO_ROOT:", REPO_ROOT)
print("CLEANED_PATH exists?", CLEANED_PATH.exists())
print("PROCESSED_DIR:", PROCESSED_DIR)
print("MAPPINGS_DIR:", MAPPINGS_DIR)

REPO_ROOT: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale
CLEANED_PATH exists? True
PROCESSED_DIR: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed
MAPPINGS_DIR: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/notebook/mappings


In [2]:
# Carico il cleaned file in modo robusto (se esiste la colonna Date la converto)
# Se CLEANED_PATH non è presente, intervenire prima di eseguire le celle successive.
cols0 = pd.read_csv(CLEANED_PATH, nrows=0).columns.tolist()
parse_arg = ['Date'] if 'Date' in cols0 else None

df = pd.read_csv(CLEANED_PATH, parse_dates=parse_arg, low_memory=False)
print("Caricato df shape:", df.shape)
print("Colonne presenti:", df.columns.tolist())
display(df.head(3))
print("Caricato df shape:", df.shape)
print("Colonne presenti:", df.columns.tolist())
display(df.head(3))

Caricato df shape: (23906, 25)
Colonne presenti: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region', '_price_clean', 'Price', '_income_clean', 'Company_mapped', 'Dealer_Name_mapped', 'Model_mapped', 'Transmission_mapped', 'Gender_mapped', 'Customer Name_mapped', 'Dealer_Region_mapped', '_price_clean_w', '_income_clean_w']


,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,...,_income_clean,Company_mapped,Dealer_Name_mapped,Model_mapped,Transmission_mapped,Gender_mapped,Customer Name_mapped,Dealer_Region_mapped,_price_clean_w,_income_clean_w
0,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,...,13500,ford,buddy storbeck's diesel service inc,expedition,auto,male,geraldine,middletown,26000.0,13500
1,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,...,1480000,dodge,c & m motors inc,durango,auto,male,gia,aurora,19000.0,1480000
2,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,...,1035000,cadillac,capitol kia,eldorado,manual,male,gianna,greenville,31500.0,1035000


Caricato df shape: (23906, 25)
Colonne presenti: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region', '_price_clean', 'Price', '_income_clean', 'Company_mapped', 'Dealer_Name_mapped', 'Model_mapped', 'Transmission_mapped', 'Gender_mapped', 'Customer Name_mapped', 'Dealer_Region_mapped', '_price_clean_w', '_income_clean_w']


,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,...,_income_clean,Company_mapped,Dealer_Name_mapped,Model_mapped,Transmission_mapped,Gender_mapped,Customer Name_mapped,Dealer_Region_mapped,_price_clean_w,_income_clean_w
0,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,...,13500,ford,buddy storbeck's diesel service inc,expedition,auto,male,geraldine,middletown,26000.0,13500
1,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,...,1480000,dodge,c & m motors inc,durango,auto,male,gia,aurora,19000.0,1480000
2,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,...,1035000,cadillac,capitol kia,eldorado,manual,male,gianna,greenville,31500.0,1035000


## Pulizia minima e regola outlier
- Trasformo Price e Annual Income in numerici (rimuovo simboli).
- Applico outlier detection **solo su Price** usando la regola IQR (1.5 * IQR).
- Tengo due dataframe:
  - `df` = originale (con outlier) — per audit;
  - `df_no_outliers` = senza outlier sul prezzo — usato per salvare i file per Tableau.

In [3]:
# Pulisco le colonne money e le converto in numerico (mantengo la colonna originale per controllo)
def to_numeric_money(series):
    # rimuovo simboli non numerici, gestisco valori vuoti
    s = series.astype(str).str.replace(r"[^\d\.\-]", "", regex=True)
    s = s.replace("", np.nan)
    return pd.to_numeric(s, errors='coerce')

# riconosco i nomi tipici delle colonne (tolleranza a varianti)
price_cols = [c for c in df.columns if c.lower().strip() in ("price ($)","price","price_$","price($)","price ($)")]
income_cols = [c for c in df.columns if c.lower().strip() in ("annual income","annual_income","income","annualincome")]

if not price_cols:
    raise RuntimeError("Non trovo la colonna Price: controlla i nomi delle colonne nel dataset.")
PRICE_COL = price_cols[0]
INCOME_COL = income_cols[0] if income_cols else None

# creo colonne pulite
# Ho chiamato le colonne _price_clean e _income_clean per non sovrascrivere i dati originali
df["_price_clean"] = to_numeric_money(df[PRICE_COL])
if INCOME_COL:
    df["_income_clean"] = to_numeric_money(df[INCOME_COL])
else:
    df["_income_clean"] = np.nan

print("Pulizia price: valori nulli ->", df["_price_clean"].isna().sum(), 
      "| min/max ->", df["_price_clean"].min(), df["_price_clean"].max())

# Outlier detection SOLO su price (IQR)
q1 = df["_price_clean"].quantile(0.25)
q3 = df["_price_clean"].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# segnalo gli outlier in una colonna booleana; li escludo solo per i file di visualizzazione
df["_price_outlier"] = (df["_price_clean"] < lower) | (df["_price_clean"] > upper)
n_outliers = int(df["_price_outlier"].sum())
print(f"Ho identificato {n_outliers} outlier sul prezzo (IQR).")

# creo df_no_outliers usato per le esportazioni verso Tableau
df_no_outliers = df[~df["_price_outlier"]].copy()
print("Shape df_no_outliers:", df_no_outliers.shape)

Pulizia price: valori nulli -> 0 | min/max -> 1200 85800
Ho identificato 1449 outlier sul prezzo (IQR).
Shape df_no_outliers: (22457, 26)


## Tableau Prep · Sezione finale
Questa sezione produce i dataset aggregati per **Dealer_Region** necessari a costruire in Tableau una mappa del **mappa del rapporto tra reddito medio e prezzo medio per auto (income-to-price ratio; più alto = più accessibile)**.  
Output previsti:
- `dealer_ratio_for_tableau.csv` → KPI per distretto
- `dealer_ratio_for_tableau_citystate.csv` → KPI per distretto + `city_state_lookup` per geocodifica
- `dealer_ratio_by_year_for_tableau.csv` → variante per analisi temporale anno su anno

### Controlli di qualità essenziali
Si effettua un controllo sintetico su null, valori negativi e numerosità dei distretti.  
L’obiettivo è prevenire problemi in fase di visualizzazione.

In [4]:
# === QC VELOCE ===
qc = {
    "rows": len(df),
    "cols": len(df.columns),
    "null_rate_price": float(df["Price ($)"].isna().mean()),
    "null_rate_income": float(df["Annual Income"].isna().mean()),
    "neg_values_price": int((df["Price ($)"] < 0).sum()),
    "neg_values_income": int((df["Annual Income"] < 0).sum()),
    "dealer_regions": int(df["Dealer_Region"].nunique()),
}
print("QC summary:", qc)

# Normalizzazione: valori negativi non sono ammessi per prezzo e reddito
df.loc[df["Price ($)"] < 0, "Price ($)"] = np.nan
df.loc[df["Annual Income"] < 0, "Annual Income"] = np.nan

QC summary: {'rows': 23906, 'cols': 26, 'null_rate_price': 0.0, 'null_rate_income': 0.0, 'neg_values_price': 0, 'neg_values_income': 0, 'dealer_regions': 7}


### Aggregazione per Dealer_Region e calcolo del ratio
Si calcolano:
- prezzo medio, mediana prezzo
- reddito medio
- numero di osservazioni per distretto
- **price_to_income_ratio** = avg_price / avg_income
- **ratio_quantile** per una color scale discreta in mappa

Si aggiunge poi un lookup `city_state` per facilitare la geocodifica in Tableau.

In [5]:
# === PATH RESOLUTION (repo-aware) ===

from pathlib import Path
import os

# Nomi file attesi in data/processed
MAIN_NAME = "database_cleaned_2.csv"
CITYSTATE_NAME = "database_for_tableau_city_state.csv"

# Candidati BASE_DIR: se il notebook è in root o dentro /notebook
candidate_roots = [Path("."), Path("..")]

RAW_MAIN = None
CITYSTATE_FILE = None

for root in candidate_roots:
    base_proc = (root / "data" / "processed").resolve()
    base_data = (root / "data").resolve()
    candidates = [
        base_proc / MAIN_NAME,
        base_data / MAIN_NAME,
        Path("/mnt/data") / MAIN_NAME,   # fallback per ambienti remoti
    ]
    for c in candidates:
        if c.exists():
            RAW_MAIN = c
            break
    candidates_city = [
        base_proc / CITYSTATE_NAME,
        base_data / CITYSTATE_NAME,
        Path("/mnt/data") / CITYSTATE_NAME,
    ]
    for c in candidates_city:
        if c.exists():
            CITYSTATE_FILE = c
            break
    if RAW_MAIN is not None:
        break  # abbiamo abbastanza per procedere (CITYSTATE è opzionale)

if RAW_MAIN is None:
    raise FileNotFoundError(
        "database_cleaned_2.csv non trovato. Atteso in 'data/processed' (o 'data'). "
        "Verificare che il notebook sia lanciato dalla root del repo o da /notebook."
    )

print("Path risolti:")
print(" - RAW_MAIN:", RAW_MAIN)
print(" - CITYSTATE_FILE:", CITYSTATE_FILE if CITYSTATE_FILE else "non trovato (lookup opzionale)")

Path risolti:
 - RAW_MAIN: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/database_cleaned_2.csv
 - CITYSTATE_FILE: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/database_for_tableau_city_state.csv


In [6]:
# === LOAD & TYPES ===
# Scopo: caricare i dati puliti e tipizzare i campi critici usati da Tableau.

import pandas as pd
import numpy as np

df = pd.read_csv(RAW_MAIN, low_memory=False)

# Campi richiesti dalla metrica e dalla mappa
required_cols = ["Dealer_Region", "Price ($)", "Annual Income", "Date"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colonne mancanti: {missing} nel file {RAW_MAIN.name}")

# Tipi coerenti
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Price ($)"] = pd.to_numeric(df["Price ($)"], errors="coerce")
df["Annual Income"] = pd.to_numeric(df["Annual Income"], errors="coerce")

# (Opzionale) carico il lookup per city/state se disponibile
cs = None
if CITYSTATE_FILE and CITYSTATE_FILE.exists():
    cs = pd.read_csv(CITYSTATE_FILE, low_memory=False)

In [7]:
# === AGGREGATION · KPI PER MAPPA ===
# Scopo: costruire i KPI per Dealer_Region necessari alla mappa in Tableau.
# Output:
#   - agg_region: KPI per distretto (avg_price, avg_income, ratio, n_obs, quantili)
#   - agg_region_filtered: come sopra + city_state_lookup (se disponibile) e filtro su n_obs

import pandas as pd
import numpy as np
from pathlib import Path

# Safety: la base df deve essere già caricata nella sezione Load (con le colonne richieste)
required = ["Dealer_Region", "Price ($)", "Annual Income"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Mancano colonne per l'aggregazione: {missing}")

# Parametri default nel caso non siano già stati definiti sopra
try:
    MIN_OBS_PER_REGION
except NameError:
    MIN_OBS_PER_REGION = 1
try:
    N_QUANTILES
except NameError:
    N_QUANTILES = 5

# Aggregazione per distretto
agg_region = (
    df.groupby("Dealer_Region", dropna=False)
      .agg(
          avg_price=("Price ($)", "mean"),
          price_median=("Price ($)", "median"),
          avg_income=("Annual Income", "mean"),
          n_obs=("Price ($)", "count"),
      )
      .reset_index()
)

# METRICA CORRETTA: Reddito / Prezzo (maggiore = più accessibile)
agg_region["income_to_price_ratio"] = agg_region["avg_income"] / agg_region["avg_price"]
agg_region.loc[~np.isfinite(agg_region["income_to_price_ratio"]), "income_to_price_ratio"] = np.nan

# Quantili per color scale discreta (calcolati sulla nuova metrica)
try:
    agg_region["ratio_quantile"] = pd.qcut(
        agg_region["income_to_price_ratio"], q=N_QUANTILES, labels=False, duplicates="drop"
    ) + 1
except ValueError:
    agg_region["ratio_quantile"] = np.nan

# Pulizia eventuale colonna legacy per evitare confusione downstream
agg_region.drop(columns=["price_to_income_ratio"], inplace=True, errors="ignore")

# Filtro opzionale su numerosità (riduce rumore in mappa)
agg_region_filtered = agg_region.loc[agg_region["n_obs"] >= MIN_OBS_PER_REGION].copy()

agg_region_filtered = agg_region.loc[agg_region["n_obs"] >= MIN_OBS_PER_REGION].copy()
agg_region_filtered.drop(columns=["price_to_income_ratio"], inplace=True, errors="ignore")

# city_state_lookup (lookup moda per distretto) se disponibile
city_lookup = None
if "cs" in globals() and isinstance(cs, pd.DataFrame) and \
   "Dealer_Region" in cs.columns and "city_state" in cs.columns:
    base_cs = cs
elif "CITYSTATE_FILE" in globals() and CITYSTATE_FILE and Path(CITYSTATE_FILE).exists():
    base_cs = pd.read_csv(CITYSTATE_FILE, low_memory=False)
else:
    base_cs = None

if base_cs is not None and "Dealer_Region" in base_cs.columns and "city_state" in base_cs.columns:
    city_lookup = (
        base_cs.groupby("Dealer_Region", dropna=False)["city_state"]
               .agg(lambda x: x.mode().iat[0] if x.mode().size else np.nan)
               .reset_index()
               .rename(columns={"city_state": "city_state_lookup"})
    )
    agg_region_filtered = agg_region_filtered.merge(city_lookup, on="Dealer_Region", how="left")
else:
    agg_region_filtered["city_state_lookup"] = np.nan

# Controllo rapido
display(agg_region.head(5))
display(agg_region_filtered.head(5))

,Dealer_Region,avg_price,price_median,avg_income,n_obs,income_to_price_ratio,ratio_quantile
0,Aurora,28334.626837,23000.0,834341.026837,3130,29.445986,4
1,Austin,28341.603628,23801.0,809496.730593,4135,28.562136,1
2,Greenville,28180.819054,22500.0,823138.340793,3128,29.209170,2
3,Janesville,27833.350955,23000.0,827446.300183,3821,29.728591,5
4,Middletown,27856.338875,22750.0,818402.594309,3128,29.379403,3


,Dealer_Region,avg_price,price_median,avg_income,n_obs,income_to_price_ratio,ratio_quantile,city_state_lookup
0,Aurora,28334.626837,23000.0,834341.026837,3130,29.445986,4,"Aurora, USA"
1,Austin,28341.603628,23801.0,809496.730593,4135,28.562136,1,"Austin, USA"
2,Greenville,28180.819054,22500.0,823138.340793,3128,29.209170,2,"Greenville, USA"
3,Janesville,27833.350955,23000.0,827446.300183,3821,29.728591,5,"Janesville, USA"
4,Middletown,27856.338875,22750.0,818402.594309,3128,29.379403,3,"Middletown, USA"


In [8]:
# ===  INIZIALIZZARE I MAPPING (crea *_mapping.csv se mancano) ===
# Scopo (per il team):
# - Assicuriamoci che per ogni colonna categorica esista un file finale
#   `mappings/<col>_mapping.csv` con colonne: raw, canonical.
# - Se il file finale ESISTE, NON lo tocchiamo (safe).
# - Se NON esiste:
#     a) se c'è il template, lo usiamo (canonical = raw di default)
#     b) altrimenti lo generiamo dai valori unici presenti in df.
#
# Nota: i file "finali" sono quelli che useremo in applicazione (Sezione E).

from pathlib import Path
import pandas as pd

# directory mapping già impostata in setup: MAPPINGS_DIR = Path("mappings")
MAPPINGS_DIR.mkdir(exist_ok=True)

# colonne su cui gestiamo mapping
cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name']

created, skipped = [], []

# utility: normalizza una serie testuale (trim + lowercase)
def _norm(s):
    return s.astype(str).str.strip().str.lower()

for col in cat_cols:
    template_path = MAPPINGS_DIR / f"{col}_mapping_template.csv"
    final_path    = MAPPINGS_DIR / f"{col}_mapping.csv"

    if final_path.exists():
        skipped.append(final_path.name)
        continue

    # caso A: ho il template → lo uso
    if template_path.exists():
        df_map = pd.read_csv(template_path, dtype=str).fillna('')
        if not {'raw','canonical'}.issubset(df_map.columns):
            # se il template non ha le colonne giuste, lo ricostruiamo
            vals = pd.Series(sorted(_norm(df[col]).unique())) if col in df.columns else pd.Series([], dtype=str)
            df_map = pd.DataFrame({'raw': vals, 'canonical': vals})
        else:
            df_map['raw']       = _norm(df_map['raw']) if 'raw' in df_map else pd.Series([], dtype=str)
            df_map['canonical'] = _norm(df_map['canonical']) if 'canonical' in df_map else pd.Series([], dtype=str)
            # canonical vuoti → rimpiazzo con raw
            df_map.loc[df_map['canonical'] == '', 'canonical'] = df_map.loc[df_map['canonical'] == '', 'raw']
    # caso B: niente template → genero dai valori del df (se la colonna esiste)
    elif col in df.columns:
        vals  = pd.Series(sorted(_norm(df[col]).unique()))
        df_map = pd.DataFrame({'raw': vals, 'canonical': vals})
    else:
        # né template né colonna → non posso creare il mapping
        continue

    df_map = df_map[['raw','canonical']].drop_duplicates()
    df_map.to_csv(final_path, index=False)
    created.append(final_path.name)

# log compatto per i colleghi
print("📄 Mapping creati   :", created if created else "none")
print("⏭  Mapping saltati  :", skipped if skipped else "none")
print("📁 Cartella mapping :", MAPPINGS_DIR.resolve())

📄 Mapping creati   : none
⏭  Mapping saltati  : ['Company_mapping.csv', 'Dealer_Name_mapping.csv', 'Model_mapping.csv', 'Transmission_mapping.csv', 'Gender_mapping.csv', 'Customer Name_mapping.csv']
📁 Cartella mapping : /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/notebook/mappings


In [9]:
# ===  ISPEZIONE MAPPING E CORREZIONI MIRATE ===
# Scopo (per il team):
# - Apriamo i file `*_mapping.csv` finali e verifichiamo che le canonical siano sensate.
# - Applichiamo poche correzioni mirate (esempi) mantenendo un BACKUP automatico.
# - Le correzioni sono additive: non sovrascriviamo manualmente il lavoro dei colleghi.

from pathlib import Path
import pandas as pd
from shutil import copyfile

MAPPINGS_DIR = Path("mappings")
cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name']

def _norm_series(s):
    return s.astype(str).str.strip().str.lower()

def _load_mapping(path: Path) -> pd.DataFrame:
    m = pd.read_csv(path, dtype=str).fillna('')
    m['raw']       = _norm_series(m['raw'])
    m['canonical'] = _norm_series(m['canonical'])
    return m[['raw','canonical']]

# 1) Anteprima rapida dei mapping disponibili
for col in cat_cols:
    p = MAPPINGS_DIR / f"{col}_mapping.csv"
    if p.exists():
        m = _load_mapping(p)
        print(f"\n— {col} mapping → {p.name} (righe: {len(m)})")
        display(m.head(10))
    else:
        print(f"\n{col}: mapping non trovato ({p.name})")

# 2) Backup + correzioni mirate (ESEMPIO su Company)
company_map = MAPPINGS_DIR / "Company_mapping.csv"
if company_map.exists():
    backup = MAPPINGS_DIR / "Company_mapping_backup.csv"
    if not backup.exists():
        copyfile(company_map, backup)
        print("Backup creato:", backup.name)
    else:
        print("Backup già presente:", backup.name)

    # dizionario di correzioni mirate (aggiungere qui eventuali altre voci concordate)
    corrections = {
        "mercedes-b": "mercedes-benz",
        # "vw": "volkswagen",   # ESEMPIO: abilita solo se il team concorda
        # "mb": "mercedes-benz"
    }

    df_map = _load_mapping(company_map)
    applied = []
    for raw_val, new_can in corrections.items():
        mask = df_map['raw'] == raw_val
        if mask.any():
            df_map.loc[mask, 'canonical'] = new_can
            applied.append(raw_val)

    # salvataggio (solo se ho applicato qualcosa)
    if applied:
        df_map.drop_duplicates().to_csv(company_map, index=False)
        print("Correzioni applicate su Company:", applied)
        display(df_map[df_map['raw'].isin(applied)].head(10))
    else:
        print("Nessuna correzione applicata su Company (dizionario vuoto o raw non presenti).")
else:
    print("\nCompany_mapping.csv non trovato: nessuna correzione mirata eseguita.")

# 3) (Opzionale) Anteprima conteggi raw per priorità di pulizia
if 'Company' in df.columns:
    counts = _norm_series(df['Company']).value_counts()
    print("\nCompany - top (count):")
    display(counts.head(30))


— Company mapping → Company_mapping.csv (righe: 30)


,raw,canonical
0,acura,acura
1,audi,audi
2,bmw,bmw
3,buick,buick
4,cadillac,cadillac
5,chevrolet,chevrolet
6,chrysler,chrysler
7,dodge,dodge
8,ford,ford
9,honda,honda



— Dealer_Name mapping → Dealer_Name_mapping.csv (righe: 28)


,raw,canonical
0,buddy storbeck's diesel service inc,buddy storbeck's diesel service inc
1,c & m motors inc,c & m motors inc
2,capitol kia,capitol kia
3,chrysler of tri-cities,chrysler of tri-cities
4,chrysler plymouth,chrysler plymouth
5,classic chevy,classic chevy
6,clay johnson auto sales,clay johnson auto sales
7,diehl motor co inc,diehl motor co inc
8,enterprise rent a car,enterprise rent a car
9,gartner buick hyundai saab,gartner buick hyundai saab



— Model mapping → Model_mapping.csv (righe: 154)


...

,raw,canonical
0,aahil,aahil
1,aaliyah,aaliyah
2,aarav,aarav
3,aaron,aaron
4,aarya,aarya
5,aayan,aayan
6,abby,abby
7,abdal,abdal
8,abdelatif,abdelatif
9,abderramine,abderramine


Backup già presente: Company_mapping_backup.csv
Correzioni applicate su Company: ['mercedes-b']


,raw,canonical
16,mercedes-b,mercedes-benz



Company - top (count):


Company
chevrolet     1819
dodge         1671
ford          1614
volkswagen    1333
mercedes-b    1285
mitsubishi    1277
chrysler      1120
oldsmobile    1111
toyota        1110
nissan         886
mercury        874
lexus          802
pontiac        796
bmw            790
volvo          789
honda          708
acura          689
cadillac       652
plymouth       617
saturn         586
lincoln        492
audi           468
buick          439
subaru         405
jeep           363
porsche        361
hyundai        264
saab           210
infiniti       195
jaguar         180
Name: count, dtype: int64

In [11]:
# === EXPORT CSV · OUTPUT DEFINITIVI PER TABLEAU ===
# Scopo (per il team):
# - Salvare gli output per Tableau in data/processed/, con path repo-aware e senza duplicazioni.
# - Ordine corretto: normalizzazione City/Country → patch State → export.
# - 'agg_region'        = KPI per distretto
# - 'agg_region_filtered' = KPI + City/Country/State (e filtro n_obs già applicato)
#
# Nota metrica: in questo notebook usiamo
#   income_to_price_ratio = avg_income / avg_price      (più alto = più accessibile)

from pathlib import Path

# 1) Individuazione della root del repo (funziona sia se il notebook è in / che in /notebook)
here = Path.cwd().resolve()
candidates = [here, here.parent, here.parent.parent]
repo_root = None
for base in candidates:
    if (base / "data" / "processed").exists():
        repo_root = base
        break
if repo_root is None:
    repo_root = here  # fallback per ambienti atipici

OUT_DIR = repo_root / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 2) Safety check: le tabelle devono esistere
missing_objs = [name for name in ["agg_region", "agg_region_filtered"] if name not in globals()]
if missing_objs:
    raise RuntimeError(
        "Mancano oggetti per l'export: "
        + ", ".join(missing_objs)
        + ". Eseguire prima 'AGGREGATION · KPI PER MAPPA'."
    )

# 3) Normalizzazione per geocodifica (City, Country) se abbiamo il lookup
#    - Tableau Web geocodifica bene City + Country.
if "city_state_lookup" in agg_region_filtered.columns:
    tmp = (
        agg_region_filtered["city_state_lookup"]
        .astype(str)
        .str.strip()
        .str.replace(r",\s*USA$", ", United States", regex=True)
    )
    # City = parte prima della virgola
    agg_region_filtered["City"] = tmp.str.split(",").str[0].str.strip()
    # Country = parte dopo la virgola, default "United States"
    c2 = tmp.str.split(",").str[1].fillna("United States").str.strip()
    c2 = c2.replace("", "United States")
    agg_region_filtered["Country"] = c2

    # Evitiamo colonne “clean” duplicate se presenti
    if "city_state_lookup_clean" in agg_region_filtered.columns:
        agg_region_filtered.drop(columns=["city_state_lookup_clean"], inplace=True, errors="ignore")

# 4) PATCH Stati (da applicare DOPO la normalizzazione City/Country)
#    - Scopo: stabilizzare la geocodifica per città ambigue. Mappatura convenzionale City → State.
city_to_state = {
    "Aurora": "Illinois",
    "Austin": "Texas",
    "Greenville": "South Carolina",
    "Janesville": "Wisconsin",
    "Middletown": "Connecticut",
    "Pasco": "Washington",
    "Scottsdale": "Arizona",
}
if "City" in agg_region_filtered.columns:
    agg_region_filtered["State"] = agg_region_filtered["City"].map(city_to_state)
else:
    print("⚠️  'City' non presente in agg_region_filtered: verificare la normalizzazione City/Country prima della patch State.")

# 5) Scrittura file (unica, senza duplicati)
out_dist      = OUT_DIR / "dealer_ratio_for_tableau.csv"            # KPI per distretto
out_dist_city = OUT_DIR / "dealer_ratio_for_tableau_citystate.csv"  # KPI + City/Country/State

agg_region.to_csv(out_dist, index=False)
agg_region_filtered.to_csv(out_dist_city, index=False)

# 5) Scrittura file (unica, senza duplicati)
out_dist      = OUT_DIR / "dealer_ratio_for_tableau.csv"            # KPI per distretto
out_dist_city = OUT_DIR / "dealer_ratio_for_tableau_citystate.csv"  # KPI + City/Country/State

agg_region.to_csv(out_dist, index=False)
agg_region_filtered.to_csv(out_dist_city, index=False)

# 6) Log finale (audit)
print("✅ Export completato")
print("→ KPI per distretto                  :", out_dist.resolve())
print("→ KPI + geocodifica (City/State/etc.):", out_dist_city.resolve())
print("Cartella export                      :", OUT_DIR.resolve())

# === CHECK FILE FINALE ===
_test = pd.read_csv(out_dist_city, nrows=3)
print("Colonne CSV finale:", list(_test.columns))
assert "income_to_price_ratio" in _test.columns, "Manca income_to_price_ratio nel file finale!"


✅ Export completato
→ KPI per distretto                  : /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/dealer_ratio_for_tableau.csv
→ KPI + geocodifica (City/State/etc.): /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/dealer_ratio_for_tableau_citystate.csv
Cartella export                      : /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed
Colonne CSV finale: ['Dealer_Region', 'avg_price', 'price_median', 'avg_income', 'n_obs', 'income_to_price_ratio', 'ratio_quantile', 'city_state_lookup', 'City', 'Country', 'State']
